In [1]:
import requests

code = '035420'  # NAVER
code = '006800'  # 미래에셋대우
code = '005930'  # 삼성전자

url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)
res = requests.get(url)
res.encoding = 'utf-8'
res.status_code

200

In [2]:
from bs4 import BeautifulSoup
soap = BeautifulSoup(res.text, 'lxml')
el_table_navi = soap.find("table", class_="Nnavi")
el_td_last = el_table_navi.find("td", class_="pgRR")
pg_last = el_td_last.a.get('href').rsplit('&')[1]
pg_last = pg_last.split('=')[1]
pg_last = int(pg_last)
pg_last

421

In [3]:
import traceback
import pandas as pd

def parse_page(code, page):
    try:
        url = 'http://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'.format(code=code, page=page)
        res = requests.get(url)
        _soap = BeautifulSoup(res.text, 'lxml')
        _df = pd.read_html(str(_soap.find("table")), header=0)[0]
        _df = _df.dropna()
        return _df
    except Exception as e:
        traceback.print_exc()
    return None

In [4]:
parse_page(code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2019.11.04,164500.0,2500.0,165500.0,167000.0,162000.0,451988.0
2,2019.11.01,167000.0,3000.0,166000.0,168500.0,164000.0,720358.0
3,2019.10.31,164000.0,9500.0,159000.0,167000.0,157500.0,1409619.0
4,2019.10.30,154500.0,1500.0,152000.0,154500.0,151500.0,290658.0
5,2019.10.29,153000.0,4000.0,157500.0,158000.0,152000.0,321007.0
9,2019.10.28,157000.0,4500.0,152500.0,158000.0,152500.0,335238.0
10,2019.10.25,152500.0,500.0,151000.0,153500.0,151000.0,162510.0
11,2019.10.24,153000.0,500.0,151500.0,153000.0,150500.0,224666.0
12,2019.10.23,152500.0,1500.0,152000.0,153500.0,151500.0,226744.0
13,2019.10.22,154000.0,2000.0,150500.0,154500.0,150500.0,360102.0


In [5]:
import datetime
str_datefrom = datetime.datetime.strftime(datetime.datetime(year=2018, month=1, day=1), '%Y.%m.%d')
str_datefrom

'2018.01.01'

In [6]:
str_dateto = datetime.datetime.strftime(datetime.datetime.today(), '%Y.%m.%d')
str_dateto

'2019.11.04'

In [9]:
df = None
for page in range(1, pg_last+1):
    _df = parse_page(code, page)
    _df_filtered = _df[_df['날짜'] > str_datefrom]
    if df is None:
        df = _df_filtered
    else:
        df = pd.concat([df, _df_filtered])
    if len(_df) > len(_df_filtered):
        break

In [10]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
10,2018.01.08,950000.0,42000.0,915000.0,953000.0,908000.0,196338.0
11,2018.01.05,908000.0,15000.0,893000.0,918000.0,880000.0,136811.0
12,2018.01.04,893000.0,22000.0,879000.0,897000.0,873000.0,130112.0
13,2018.01.03,871000.0,14000.0,876000.0,884000.0,864000.0,84790.0
1,2018.01.02,885000.0,15000.0,871000.0,885000.0,870000.0,93587.0


In [11]:
import os
path_dir = 'data/2019-11-04-crawling'
if not os.path.exists(path_dir):
    os.makedirs(path_dir)
path = os.path.join(path_dir, '{code}_{date_from}_{date_to}.csv'.format(code=code, date_from=str_datefrom, date_to=str_dateto))
path

'data/2019-11-04-crawling\\035420_2018.01.01_2019.11.04.csv'

In [12]:
df.to_csv(path, index=False)